In [ ]:
# optimisation_maintenance_planning_pro.py

# ======= IMPORTS =======
import pulp
import numpy as np
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

# ======= 1️⃣ PARAMÈTRAGE =======

E = [
    "Moteur principal", "Pompe hydraulique", "Transmission", "Circuit de refroidissement",
    "Lubrification moteur", "PTO avant", "Circuit de freinage", "Direction hydraulique",
    "Régime moteur", "Essieux avant"
]

S = ["Révision", "Remplacement", "Contrôle", "Vidange", "Graissage", "Lubrification", "Amélioration", "Inspection"]

P = ["Technicien A", "Technicien B", "Technicien C"]

R = ["Piece1", "Piece2", "Piece3"]

T = list(range(1, 25))  # Jours 1 → 24

# RUL donné
rul_array = np.array([
    0.5194773,  0.5286141,  0.5256151,  0.52865183, 0.5280742,  0.52900714, 
    0.52648014, 0.5291931,  0.5270387,  0.5344134,  0.53038186, 0.53363174,
    0.53024626, 0.53480387, 0.53929555, 0.5377515,  0.5420548,  0.54542494,
    0.54888,    0.54889727, 0.55220103, 0.5603916,  0.5576258,  0.56187624
])

# Seuil critique
D = { (e,t): 0.53 for e in E for t in T }
RUL = { (e, t): rul_array[t-1] for e in E for t in T }
beta = { (e,t): int(RUL[(e,t)] <= D[(e,t)]) for e in E for t in T }

# ======= PARAMÈTRES FIXES =======
# Nombre max de techniciens par compétence et par jour (exemple : 2 partout)
N_s_t = { (s, t): 2 for s in S for t in T }
# Durée des tâches (tau) en heures → par logique métier
tau = {
    (e, s, p): {
        "Révision": 3,
        "Remplacement": 4,
        "Contrôle": 2,
        "Vidange": 1,
        "Graissage": 1,
        "Lubrification": 2,
        "Amélioration": 5,
        "Inspection": 2
    }[s]
    for e in E for s in S for p in P
}
# Temps de travail dispo par technicien (8h/jour par défaut)
phi = { p: 8 for p in P }
# Matrice de qualification des techniciens (alpha) → logique :
alpha = {
    (s, p): 1 if (
        (s in ["Révision", "Remplacement", "Contrôle", "Inspection"]) or
        (s in ["Vidange", "Graissage", "Lubrification"] and p in ["Technicien A", "Technicien B"]) or
        (s == "Amélioration" and p in ["Technicien C"])
    ) else 0
    for s in S for p in P
}

# Délai max pour maintenance critique (2 jours pour tous composants)
delta = { e: 2 for e in E }
# Matrice kappa → pièces nécessaires par composant et type de maintenance :
# Exemple logique :
pieces_par_composant = {
    "Moteur principal": ["Piece1", "Piece2"],
    "Pompe hydraulique": ["Piece1", "Piece3"],
    "Transmission": ["Piece2", "Piece3"],
    "Circuit de refroidissement": ["Piece1", "Piece2"],
    "Lubrification moteur": ["Piece1", "Piece3"],
    "PTO avant": ["Piece2", "Piece3"],
    "Circuit de freinage": ["Piece1", "Piece2"],
    "Direction hydraulique": ["Piece1", "Piece3"],
    "Régime moteur": ["Piece3"],
    "Essieux avant": ["Piece2", "Piece3"]
}
kappa = {}
for e in E:
    for s in S:
        for r in R:
            if r in pieces_par_composant[e]:
                kappa[(e, s, r)] = 1
            else:
                kappa[(e, s, r)] = 0
# Stock disponible de chaque pièce par jour (exemple : 5 pièces/jour partout)
M_r_t = { (r, t): 5 for r in R for t in T }
# Criticité gamma par logique (plus critique pour Amélioration, Remplacement)
gamma = {}
for e in E:
    for s in S:
        if s in ["Amélioration", "Remplacement"]:
            gamma[(e, s)] = 1.8
        elif s in ["Révision", "Contrôle"]:
            gamma[(e, s)] = 1.5
        elif s in ["Vidange", "Graissage", "Lubrification"]:
            gamma[(e, s)] = 1.2
        else:  # Inspection
            gamma[(e, s)] = 1.0
# Coûts main d’œuvre (C_main) → techniciens spécialisés ont un coût un peu plus élevé
C_main = {}
for e in E:
    for s in S:
        for p in P:
            if p == "Technicien A":
                C_main[(e, s, p)] = 60
            elif p == "Technicien B":
                C_main[(e, s, p)] = 65
            else:  # Technicien C
                C_main[(e, s, p)] = 70
# Coûts des pièces (C_piece) → par logique :
C_piece = {}
for e in E:
    for s in S:
        if s in ["Remplacement", "Amélioration"]:
            C_piece[(e, s)] = 40
        elif s in ["Révision", "Contrôle"]:
            C_piece[(e, s)] = 30
        else:
            C_piece[(e, s)] = 20

# Coûts sous-traitance (C_sous) → élevé pour Amélioration, moyen pour Remplacement
C_sous = {}
for e in E:
    for s in S:
        if s == "Amélioration":
            C_sous[(e, s)] = 140
        elif s == "Remplacement":
            C_sous[(e, s)] = 120
        elif s in ["Révision", "Contrôle"]:
            C_sous[(e, s)] = 100
        else:
            C_sous[(e, s)] = 80


# ======= 2️⃣ CONSTRUCTION DU MODÈLE =======

model = pulp.LpProblem("Optimisation_Maintenance", pulp.LpMinimize)

x = pulp.LpVariable.dicts("x", ((e,s,p,t) for e in E for s in S for p in P for t in T), cat='Binary')
z = pulp.LpVariable.dicts("z", ((e,s,t) for e in E for s in S for t in T), cat='Binary')

# Fonction Objectif
model += pulp.lpSum(
    gamma[(e,s)] * (
        pulp.lpSum(C_main[(e,s,p)] * x[(e,s,p,t)] for p in P)
        + C_piece[(e,s)] * z[(e,s,t)]
        + C_sous[(e,s)] * z[(e,s,t)]
    )
    for e in E for s in S for t in T
)

# Contraintes
for e in E:
    for s in S:
        for t in T:
            model += pulp.lpSum(alpha[(s,p)] * x[(e,s,p,t)] for p in P) == z[(e,s,t)], f"Assign_{e}_{s}_{t}"

for p in P:
    for t in T:
        model += pulp.lpSum(tau[(e,s,p)] * x[(e,s,p,t)] for e in E for s in S) <= phi[p], f"WorkTime_{p}_{t}"

for s in S:
    for t in T:
        model += pulp.lpSum(x[(e,s,p,t)] for e in E for p in P) <= N_s_t[(s,t)], f"MaxEmp_{s}_{t}"

for e in E:
    for s in S:
        for t in T:
            if beta[(e,t)] == 1:
                t_window = [t_prime for t_prime in T if t_prime >= t and t_prime <= t + delta[e]]
                model += pulp.lpSum(z[(e,s,t_prime)] for t_prime in t_window) == 1, f"CriticalMaint_{e}_{s}_{t}"

for r in R:
    for t in T:
        model += pulp.lpSum(kappa[(e,s,r)] * z[(e,s,t)] for e in E for s in S) <= M_r_t[(r,t)], f"Stock_{r}_{t}"

# ======= 3️⃣ RÉSOLUTION =======

solver = pulp.PULP_CBC_CMD(msg=True)
model.solve(solver)

# ======= 4️⃣ EXTRACTION DES RÉSULTATS =======

print("\n===== RÉSULTATS =====\n")
print(f"Statut de résolution: {pulp.LpStatus[model.status]}")
print(f"Coût total optimal = {pulp.value(model.objective):.2f}\n")

# Planning maintenance
planning_maint = []
for e in E:
    for s in S:
        for t in T:
            if pulp.value(z[(e,s,t)]) > 0.5:
                planning_maint.append({
                    'Composant': e,
                    'Maintenance': s,
                    'Jour': t
                })

# Planning technicien
planning_tech = []
for e in E:
    for s in S:
        for p in P:
            for t in T:
                if pulp.value(x[(e,s,p,t)]) > 0.5:
                    planning_tech.append({
                        'Technicien': p,
                        'Composant': e,
                        'Maintenance': s,
                        'Jour': t
                    })

# Affichage pro des techniciens
print("\n👷 PLANNING DES TECHNICIENS (TEXTE) :\n")
for row in sorted(planning_tech, key=lambda x: (x['Technicien'], x['Jour'])):
    print(f"Technicien: {row['Technicien']:<12} | Jour: {row['Jour']:>2} | Maintenance: {row['Maintenance']:<15} | Composant: {row['Composant']}")

# ======= 5️⃣ PLOTTING PLOTLY GANTT PRO =======

# Créer dataframe GANTT techniciens
df_gantt = []

for row in planning_tech:
    df_gantt.append({
        'Technicien': row['Technicien'],
        'Composant': row['Composant'],
        'Maintenance': row['Maintenance'],
        'Jour': row['Jour'],
        'Start': pd.to_datetime("2025-04-01") + pd.to_timedelta(row['Jour']-1, unit='D'),
        'Finish': pd.to_datetime("2025-04-01") + pd.to_timedelta(row['Jour'], unit='D')
    })

df_gantt = pd.DataFrame(df_gantt)

# Plot GANTT
fig = px.timeline(
    df_gantt,
    x_start="Start",
    x_end="Finish",
    y="Composant",
    color="Maintenance",
    text="Technicien",
    hover_data=["Maintenance", "Start", "Finish", "Composant", "Technicien"],
    title="📅 Planning Maintenance par Technicien (GANTT PRO)"
)

fig.update_yaxes(autorange="reversed")
fig.update_traces(textposition='inside', insidetextanchor='middle')

fig.update_layout(
    font=dict(size=12),
    xaxis_title="Date",
    yaxis_title="Composant",
    legend_title="Maintenance",
    plot_bgcolor='white'
)

# Afficher
fig.show()

# Optionnel : Sauvegarder PNG (pour rapport)
# fig.write_image("Planning_Techniciens_GANTT.png", scale=2)



===== RÉSULTATS =====

Statut de résolution: Infeasible
Coût total optimal = 34096.49


👷 PLANNING DES TECHNICIENS (TEXTE) :

Technicien: Technicien A | Jour:  1 | Maintenance: Graissage       | Composant: Moteur principal
Technicien: Technicien A | Jour:  1 | Maintenance: Graissage       | Composant: Circuit de refroidissement
Technicien: Technicien A | Jour:  1 | Maintenance: Contrôle        | Composant: Circuit de freinage
Technicien: Technicien A | Jour:  1 | Maintenance: Vidange         | Composant: Circuit de freinage
Technicien: Technicien A | Jour:  1 | Maintenance: Lubrification   | Composant: Régime moteur
Technicien: Technicien A | Jour:  2 | Maintenance: Vidange         | Composant: Pompe hydraulique
Technicien: Technicien A | Jour:  2 | Maintenance: Vidange         | Composant: PTO avant
Technicien: Technicien A | Jour:  2 | Maintenance: Lubrification   | Composant: Circuit de freinage
Technicien: Technicien A | Jour:  2 | Maintenance: Inspection      | Composant: Circuit

In [9]:
import pulp
import numpy as np
import random

# ==========================
# Données de base
# ==========================

# Ensemble des composants / machines
E = [
    "Moteur principal", "Pompe hydraulique", "Transmission", "Circuit de refroidissement",
    "Lubrification moteur", "PTO avant", "Circuit de freinage", "Direction hydraulique",
    "Régime moteur", "Essieux avant"
]

# Types de maintenance
S = ["Révision", "Remplacement", "Contrôle", "Vidange", "Graissage", "Lubrification", "Amélioration", "Inspection"]

# Techniciens
P = ["Technicien A", "Technicien B", "Technicien C"]

# Pièces détachées
R = ["Piece1", "Piece2", "Piece3"]

# Horizon temporel (jours 1 à 24)
T = list(range(1, 25))

# Ensemble des scénarios
Omega = ["omega1", "omega2", "omega3"]
p_omega = { omega: 1 / len(Omega) for omega in Omega }

# ==========================
# Paramètres RUL, seuils, beta
# ==========================
rul_array = np.array([
    0.5194773,  0.5286141,  0.5256151,  0.52865183, 0.5280742,  0.52900714,
    0.52648014, 0.5291931,  0.5270387,  0.5344134,  0.53038186, 0.53363174,
    0.53024626, 0.53480387, 0.53929555, 0.5377515,  0.5420548,  0.54542494,
    0.54888,    0.54889727, 0.55220103, 0.5603916,  0.5576258,  0.56187624
])
D = { (e,t): 0.53 for e in E for t in T }
RUL = { (e, t): rul_array[t-1] for e in E for t in T }
beta = { (e,t): int(RUL[(e,t)] <= D[(e,t)]) for e in E for t in T }

# ==========================
# Paramètres fixes
# ==========================
N_s_t = { (s, t): 2 for s in S for t in T }

tau = {
    (e, s, p): {
        "Révision": 3,
        "Remplacement": 4,
        "Contrôle": 2,
        "Vidange": 1,
        "Graissage": 1,
        "Lubrification": 2,
        "Amélioration": 5,
        "Inspection": 2
    }[s]
    for e in E for s in S for p in P
}

phi = { p: 8 for p in P }

alpha = {
    (s, p): 1 if (
        (s in ["Révision", "Remplacement", "Contrôle", "Inspection"]) or
        (s in ["Vidange", "Graissage", "Lubrification"] and p in ["Technicien A", "Technicien B"]) or
        (s == "Amélioration" and p in ["Technicien C"])
    ) else 0
    for s in S for p in P
}

delta = { e: 2 for e in E }

pieces_par_composant = {
    "Moteur principal": ["Piece1", "Piece2"],
    "Pompe hydraulique": ["Piece1", "Piece3"],
    "Transmission": ["Piece2", "Piece3"],
    "Circuit de refroidissement": ["Piece1", "Piece2"],
    "Lubrification moteur": ["Piece1", "Piece3"],
    "PTO avant": ["Piece2", "Piece3"],
    "Circuit de freinage": ["Piece1", "Piece2"],
    "Direction hydraulique": ["Piece1", "Piece3"],
    "Régime moteur": ["Piece3"],
    "Essieux avant": ["Piece2", "Piece3"]
}

kappa = {}
for e in E:
    for s in S:
        for r in R:
            kappa[(e, s, r)] = 1 if r in pieces_par_composant[e] else 0

# Stock M_r_t(omega) par scénario
M_r_t_omega = { (r, t, omega): max(3, int(random.gauss(5, 1))) for r in R for t in T for omega in Omega }

gamma = {}
for e in E:
    for s in S:
        if s in ["Amélioration", "Remplacement"]:
            gamma[(e, s)] = 1.8
        elif s in ["Révision", "Contrôle"]:
            gamma[(e, s)] = 1.5
        elif s in ["Vidange", "Graissage", "Lubrification"]:
            gamma[(e, s)] = 1.2
        else:
            gamma[(e, s)] = 1.0

C_main = {}
for e in E:
    for s in S:
        for p in P:
            C_main[(e, s, p)] = 60 if p == "Technicien A" else (65 if p == "Technicien B" else 70)

C_piece = {}
for e in E:
    for s in S:
        if s in ["Remplacement", "Amélioration"]:
            C_piece[(e, s)] = 40
        elif s in ["Révision", "Contrôle"]:
            C_piece[(e, s)] = 30
        else:
            C_piece[(e, s)] = 20

C_sous = {}
for e in E:
    for s in S:
        if s == "Amélioration":
            C_sous[(e, s)] = 140
        elif s == "Remplacement":
            C_sous[(e, s)] = 120
        elif s in ["Révision", "Contrôle"]:
            C_sous[(e, s)] = 100
        else:
            C_sous[(e, s)] = 80

C_achat = { r: 50 for r in R }

# ==========================
# Modèle PuLP
# ==========================
model = pulp.LpProblem("Optimisation_Maintenance_Stochastique", pulp.LpMinimize)

# Variables
x = pulp.LpVariable.dicts("x", ((e,s,p,t) for e in E for s in S for p in P for t in T), cat="Binary")
z = pulp.LpVariable.dicts("z", ((e,s,t) for e in E for s in S for t in T), cat="Binary")
u = pulp.LpVariable.dicts("u", ((r,t,omega) for r in R for t in T for omega in Omega), lowBound=0, cat="Continuous")

# Objectif
objective = pulp.lpSum(
    p_omega[omega] * (
        pulp.lpSum(
            gamma[(e,s)] * (
                pulp.lpSum( C_main[(e,s,p)] * x[(e,s,p,t)] for p in P )
                + C_piece[(e,s)] * z[(e,s,t)]
                + C_sous[(e,s)] * z[(e,s,t)]
            )
            for e in E for s in S for t in T
        )
        + pulp.lpSum( C_achat[r] * u[(r,t,omega)] for r in R for t in T )
    )
    for omega in Omega
)
model += objective

# Contraintes
# Assignation employés
for e in E:
    for s in S:
        for t in T:
            model += pulp.lpSum( alpha[(s,p)] * x[(e,s,p,t)] for p in P ) == z[(e,s,t)], f"assign_{e}_{s}_{t}"

# Temps de travail
for p in P:
    for t in T:
        model += pulp.lpSum( tau[(e,s,p)] * x[(e,s,p,t)] for e in E for s in S ) <= phi[p], f"time_{p}_{t}"

# Nombre max de techniciens
for s in S:
    for t in T:
        model += pulp.lpSum( x[(e,s,p,t)] for e in E for p in P ) <= N_s_t[(s,t)], f"nb_tech_{s}_{t}"

# Maintenance obligatoire RUL critique
for e in E:
    for s in S:
        for t in T:
            if beta[(e,t)] == 1:
                window = range(t, min(t + delta[e] + 1, max(T)+1))
                model += pulp.lpSum( z[(e,s,tp)] for tp in window ) >= 1, f"RUL_critique_{e}_{s}_{t}"

# Contraintes de stock par scénario
for r in R:
    for t in T:
        for omega in Omega:
            model += pulp.lpSum( kappa[(e,s,r)] * z[(e,s,t)] for e in E for s in S ) <= M_r_t_omega[(r,t,omega)] + u[(r,t,omega)], f"stock_{r}_{t}_{omega}"

# Résolution
solver = pulp.PULP_CBC_CMD(msg=True)
model.solve(solver)

# Résultats
print(f"Status : {pulp.LpStatus[model.status]}")
print(f"Coût total optimisé : {pulp.value(model.objective)}")

# Exemple de sorties
for e in E:
    for s in S:
        for t in T:
            if pulp.value(z[(e,s,t)]) > 0.5:
                print(f">>> Maintenance planifiée : {s} sur {e} au jour {t}")


Status : Infeasible
Coût total optimisé : 73988.33333325839
>>> Maintenance planifiée : Révision sur Moteur principal au jour 3
>>> Maintenance planifiée : Révision sur Moteur principal au jour 6
>>> Maintenance planifiée : Révision sur Moteur principal au jour 9
>>> Maintenance planifiée : Remplacement sur Moteur principal au jour 3
>>> Maintenance planifiée : Remplacement sur Moteur principal au jour 6
>>> Maintenance planifiée : Remplacement sur Moteur principal au jour 9
>>> Maintenance planifiée : Contrôle sur Moteur principal au jour 3
>>> Maintenance planifiée : Contrôle sur Moteur principal au jour 6
>>> Maintenance planifiée : Contrôle sur Moteur principal au jour 9
>>> Maintenance planifiée : Vidange sur Moteur principal au jour 3
>>> Maintenance planifiée : Vidange sur Moteur principal au jour 6
>>> Maintenance planifiée : Vidange sur Moteur principal au jour 9
>>> Maintenance planifiée : Graissage sur Moteur principal au jour 1
>>> Maintenance planifiée : Graissage sur Mote

In [12]:
import pandas as pd
import plotly.express as px

# On prépare les données "type Gantt enrichi"
gantt_data = []

for e in E:
    for s in S:
        for t in T:
            if pulp.value(z[(e, s, t)]) > 0.5:
                # Chercher le technicien affecté (x=1)
                assigned_techs = [p for p in P if pulp.value(x[(e, s, p, t)]) > 0.5]
                tech_str = ", ".join(assigned_techs) if assigned_techs else "Non assigné"
                
                gantt_data.append({
                    'Composant': e,
                    'Maintenance': s,
                    'Technicien': tech_str,
                    'Jour': t,
                    'Start': pd.Timestamp('2025-04-01') + pd.to_timedelta(t - 1, unit='D'),
                    'Finish': pd.Timestamp('2025-04-01') + pd.to_timedelta(t, unit='D')
                })

# DataFrame Gantt
df_gantt = pd.DataFrame(gantt_data)

# Créer le Gantt interactif
fig = px.timeline(
    df_gantt,
    x_start="Start",
    x_end="Finish",
    y="Composant",
    color="Maintenance",
    text="Technicien",
    category_orders={"Composant": list(reversed(df_gantt['Composant'].unique()))}
)

# Ajustements
fig.update_yaxes(title="Composant")
fig.update_xaxes(title="Date")
fig.update_layout(
    title="Planification optimisée des maintenances (Modèle stochastique)",
    legend_title="Maintenance",
    xaxis_tickformat='%b %d, %Y'
)

# Afficher le texte dans les barres
fig.update_traces(insidetextanchor="middle", textposition='inside', textfont_size=10)

# Affichage interactif
fig.show()


In [13]:
# Préparer les données achat supplémentaire u
achat_data = []

for omega in Omega:
    for r in R:
        for t in T:
            val = pulp.value(u[(r, t, omega)])
            if val > 0.01:  # seuil pour n'afficher que les achats réels
                achat_data.append({
                    'Scénario': omega,
                    'Pièce': r,
                    'Jour': t,
                    'Achat_supplementaire': val
                })

# Créer DataFrame
df_achat = pd.DataFrame(achat_data)

# Vérif
print(df_achat.head())


  Scénario   Pièce  Jour  Achat_supplementaire
0   omega1  Piece1     1                   2.4
1   omega1  Piece1     2                   2.0
2   omega1  Piece1     3                  27.6
3   omega1  Piece1     4                   3.4
4   omega1  Piece1     5                   2.0


In [24]:
import plotly.express as px

fig = px.bar(
    df_achat,
    x='Jour',
    y='Achat_supplementaire',
    color='Pièce',
    facet_row='Scénario',
    barmode='group',
    title="Analyse des achats supplémentaires de pièces selon les scénarios d'incertitude",
    labels={
        'Achat_supplementaire': 'Nombre de pièces achetées',
        'Jour': 'Jour de planification'
    },
    text='Achat_supplementaire'
)

# Layout professionnel
fig.update_layout(
    height=850,
    title_font_size=20,
    font=dict(size=14),
    legend_title_text="Pièce",
    xaxis_title="Jour",
    plot_bgcolor='white',
    paper_bgcolor='white',
    bargap=0.25,
    bargroupgap=0.15
)

# Texte sur les barres
fig.update_traces(
    texttemplate='%{text:.0f}',
    textposition='outside'
)

# Nettoyer les titres des sous-graphiques → juste omega1, omega2, omega3
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1], font_size=14, font_color="black"))

# Label Y seulement sur la première ligne (row=1)
fig.update_yaxes(title_text="Nombre de pièces supplémentaires", row=2, col=1)

# Supprimer le Y label sur les autres lignes (row=2 et row=3 ici)
fig.update_yaxes(title_text="", row=1, col=1)
fig.update_yaxes(title_text="", row=3, col=1)

# Affichage final
fig.show()


In [29]:
import pandas as pd
import plotly.express as px

# ==== 1️⃣ Calcul des coûts ====

# Coût commun (z, x)
cout_fixe = 0
for e in E:
    for s in S:
        for t in T:
            cout_fixe += gamma[(e,s)] * (
                sum(C_main[(e,s,p)] * pulp.value(x[(e,s,p,t)]) for p in P)
                + C_piece[(e,s)] * pulp.value(z[(e,s,t)])
                + C_sous[(e,s)] * pulp.value(z[(e,s,t)])
            )

# Coût total par scénario ω
cout_total_omega = {}
cout_achat_omega = {}

for omega in Omega:
    cout_u = 0
    for r in R:
        for t in T:
            cout_u += C_achat[r] * pulp.value(u[(r,t,omega)])
    
    cout_total = cout_fixe + cout_u
    cout_total_omega[omega] = cout_total
    cout_achat_omega[omega] = cout_u

# ==== 2️⃣ Construire le DataFrame ====

df_cout = pd.DataFrame({
    'Scénario': list(cout_total_omega.keys()),
    'Coût_fixe (DH)': [cout_fixe for _ in Omega],
    'Coût_achats_suppl (DH)': list(cout_achat_omega.values()),
    'Coût_total (DH)': list(cout_total_omega.values())
})

# Arrondir pour l'affichage
df_cout['Coût_fixe (DH)'] = df_cout['Coût_fixe (DH)'].round(2)
df_cout['Coût_achats_suppl (DH)'] = df_cout['Coût_achats_suppl (DH)'].round(2)
df_cout['Coût_total (DH)'] = df_cout['Coût_total (DH)'].round(2)

# ==== 3️⃣ Afficher la table ====

print("\n=== Tableau des coûts par scénario ===")
print(df_cout)

# ==== 4️⃣ Graphique comparatif des coûts ====

fig = px.bar(
    df_cout,
    x='Scénario',
    y='Coût_total (DH)',
    text='Coût_total (DH)',
    title="Comparaison des coûts totaux par scénario (Modèle stochastique)",
    labels={'Coût_total (DH)': 'Coût total (DH)'}
)

# Améliorer l'esthétique
fig.update_traces(texttemplate='%{text:.2f}', textposition='outside')
fig.update_layout(
    height=600,
    title_font_size=20,
    font=dict(size=14),
    plot_bgcolor='white',
    paper_bgcolor='white',
    bargap=0.3
)

# Afficher le graphe
fig.show()



=== Tableau des coûts par scénario ===
  Scénario  Coût_fixe (DH)  Coût_achats_suppl (DH)  Coût_total (DH)
0   omega1         57535.0                 16470.0          74005.0
1   omega2         57535.0                 16120.0          73655.0
2   omega3         57535.0                 16770.0          74305.0
